In [24]:
#Kaggle Insurance Claim
# -*- coding: utf-8 -*-
#This is the current one 
"""
Created on Tue Nov 14 11:14:54 2017

@author: Santanu P
"""

from __future__ import print_function
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn import preprocessing


train = pd.read_csv("/home/santanu/Downloads/Insurance Claim/train.csv")
test = pd.read_csv("/home/santanu/Downloads/Insurance Claim/test.csv")

features =[
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]

comb = [['ps_reg_01', 'ps_car_02_cat'],['ps_reg_01', 'ps_car_04_cat']]
feat_comb = []
for  c in comb:
    f1 = c[0]
    f2 = c[1]
    comb_f = str(f1) + '_' + str(f2)
    train[comb_f] = train.apply(lambda row:str(row[f1]) + '_' + str(row[f2]),axis=1)
    test[comb_f] = test.apply(lambda row:str(row[f1]) + '_' + str(row[f2]),axis=1)
    feat_comb.append(comb_f)
    
for f in feat_comb:
    features.append(f)


cat_feat = [
   "ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_04_cat",
    "ps_reg_01_ps_car_02_cat",
    "ps_reg_01_ps_car_04_cat" ]  
	

bin_feat = [
    "ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin"]


for f in cat_feat:
    print(f)
    le = preprocessing.LabelEncoder()
    le.fit(train[f].append(test[f]))
    train[f] = le.transform(train[f])
    test[f] = le.transform(test[f])

L1 = len(train)
L2 = len(test) 
pd_dummies = pd.get_dummies(train.append(test)[features],columns=cat_feat)
train = pd_dummies[0:L1]
test   = pd_dummies[L1:]


feat_bin_cat_ex = []
for f in features:
    if f not in (cat_feat + bin_feat):
        feat_bin_cat_ex.append(f)

for f in feat_bin_cat_ex:
    median = np.median(train[train[f] != -1][f].values)
    train[f] = train[f].apply(lambda x:median if x == -1 else x)
    test[f] = test[f].apply(lambda x:median if x == -1 else x)

for f in feat_bin_cat_ex:
    mean = np.mean(train[f].values)
    std = np.std(train[f].values)
    train[f] = train[f].apply(lambda x: (x - mean)/float(std))
    test[f] =  test[f].apply(lambda x: (x - mean)/float(std))

features_new = list(test.columns)
train_x1 = train[features_new].values
train_y1 = train['target'].values
test_x1 = test[features_new].values
print(np.shape(train_x1))
print(np.shape(test_x1))

ps_ind_05_cat
ps_car_01_cat
ps_car_07_cat
ps_car_03_cat
ps_car_06_cat
ps_car_04_cat
ps_car_09_cat
ps_car_02_cat
ps_ind_02_cat
ps_car_05_cat
ps_car_08_cat
ps_ind_04_cat
ps_reg_01_ps_car_02_cat
ps_reg_01_ps_car_04_cat


/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/santanu/tensorflowGPU/lib/python2.7/site-packages/ipykernel/__main__.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

(595212, 224)
(892816, 224)


In [29]:
## Neural Network with 0.282 leaderboard score
df_pred = pd.DataFrame()
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn import preprocessing
import tensorflow
import keras
from keras.datasets import mnist
from keras.models import Sequential
#from keras.layers import Dense, Dropout,PReLU
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback
from sklearn.metrics import log_loss, roc_auc_score
import theano
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import custom_object_scope
from keras import callbacks

from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

class roc_auc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.x, verbose=0)
        roc = roc_auc_score(self.y, y_pred)
        logs['roc_auc'] = roc_auc_score(self.y, y_pred)
        logs['norm_gini'] = ( roc_auc_score(self.y, y_pred) * 2 ) - 1

        y_pred_val = self.model.predict_proba(self.x_val, verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        logs['roc_auc_val'] = roc_auc_score(self.y_val, y_pred_val)
        logs['norm_gini_val'] = ( roc_auc_score(self.y_val, y_pred_val) * 2 ) - 1

        print('\rroc_auc: %s - roc_auc_val: %s - norm_gini: %s - norm_gini_val: %s' % (str(round(roc,5)),str(round(roc_val,5)),str(round((roc*2-1),5)),str(round((roc_val*2-1),5))), end=10*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

  

train = pd.read_csv("/home/santanu/Downloads/Insurance Claim/train.csv")
train_y1 = train['target'].values
pred_nn = []
pred_df = pd.DataFrame()
#k = 0 
k = 0 
run = 0

kf = KFold(n_splits=5, random_state=0, shuffle=True)
for train_index, test_index in kf.split(train_x1):
    k += 1 
    test_df = test.copy()
    X_train,X_test = train_x1[train_index], train_x1[test_index]
    y_train, y_test = train_y1[train_index], train_y1[test_index]
    
    batch_size = 4096
    epochs = 300
    model = Sequential()
        #256
    model.add(Dense(512,input_shape=(224,)))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
        #256
    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    #sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) 
    model.compile(loss='binary_crossentropy', optimizer=adam,metrics=['accuracy'])
    class_weight = {1 :26,
          0:1}

    callbacks = [
                roc_auc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test)),  # call this before EarlyStopping
                EarlyStopping(monitor='norm_gini_val', patience=3, mode='max', verbose=1),
             CSVLogger('keras-5fold-run-01-v1-epochs.log', separator=',', append=False),
                ModelCheckpoint(
                        'keras-5fold-run-01-v1-fold-' + str('%02d' % (k + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check',
                        monitor='norm_gini_val', mode='max', # mode must be set to max or Keras will be confused
                        save_best_only=True,
                        verbose=1)
            ]

    history = model.fit(X_train, y_train,
                            batch_size=batch_size,
                            class_weight = class_weight,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(X_test,y_test),callbacks=callbacks)
    del model
    model = keras.models.load_model('keras-5fold-run-01-v1-fold-' + str('%02d' % (k + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check')
    score = model.evaluate(X_test, y_test, verbose=0)
        #model_name = '/home/santanu/Downloads/Insurance Claim/' + 'nn_model__t_' + str(k) 
    model_name = '/home/santanu/Downloads/Insurance Claim/' + 'nn_model__tt_an' + str(k) 

    model.save(model_name)

    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    pred = model.predict(X_test,batch_size=5000)
    pred_nn = pred_nn + pred[:,0].tolist()
    pred = model.predict(test_x1,batch_size=5000)
    df_pred[str(k)] = pred[:,0].tolist()
    #print(eval_gini(y_test,pred[:,0].tolist()))
         


pred_df['nn'] = pred_nn
pred_df.to_csv('/home/santanu/Downloads/Insurance Claim/val_pred__ttt_an.csv')
pred_new = np.mean(df_pred[['1','2','3','4','5']].values,axis=1)
df_pred['target'] = pred_new
test_1 = pd.read_csv("/home/santanu/Downloads/Insurance Claim/test.csv")
df_pred['id'] = test_1['id']
df_pred[['id','target']].to_csv("/home/santanu/Downloads/Insurance Claim/test_subtt_an.csv",index=False)





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 512)               115200    
_________________________________________________________________
p_re_lu_7 (PReLU)            (None, 512)               512       
_________________________________________________________________
batch_normalization_7 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
_________________________________________________________________
p_re_lu_8 (PReLU)            (None, 256)               256       
_________________________________________________________________
batch_normalization_8 (Batch (None, 256)               1024      
__________

KeyError: 'id'

In [33]:
df_pred = pd.DataFrame()
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn import preprocessing
import tensorflow
import keras
from keras.datasets import mnist
from keras.models import Sequential
#from keras.layers import Dense, Dropout,PReLU
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback
from sklearn.metrics import log_loss, roc_auc_score
import theano
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import custom_object_scope
from keras import callbacks

from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

class roc_auc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.x, verbose=0)
        roc = roc_auc_score(self.y, y_pred)
        logs['roc_auc'] = roc_auc_score(self.y, y_pred)
        logs['norm_gini'] = ( roc_auc_score(self.y, y_pred) * 2 ) - 1

        y_pred_val = self.model.predict_proba(self.x_val, verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        logs['roc_auc_val'] = roc_auc_score(self.y_val, y_pred_val)
        logs['norm_gini_val'] = ( roc_auc_score(self.y_val, y_pred_val) * 2 ) - 1

        print('\rroc_auc: %s - roc_auc_val: %s - norm_gini: %s - norm_gini_val: %s' % (str(round(roc,5)),str(round(roc_val,5)),str(round((roc*2-1),5)),str(round((roc_val*2-1),5))), end=10*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

  

train = pd.read_csv("/home/santanu/Downloads/Insurance Claim/train.csv")
train_y1 = train['target'].values
pred_nn = []
pred_df = pd.DataFrame()
#k = 0 
k = 0 
run = 0

kf = KFold(n_splits=5, random_state=0, shuffle=True)
for train_index, test_index in kf.split(train_x1):
    k += 1 
    test_df = test.copy()
    X_train,X_test = train_x1[train_index], train_x1[test_index]
    y_train, y_test = train_y1[train_index], train_y1[test_index]
    
    batch_size = 4096
    epochs = 300
    model = Sequential()
        #256
    model.add(Dense(400,input_shape=(224,)))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
        #256
    model.add(Dense(200))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    #sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) 
    model.compile(loss='binary_crossentropy', optimizer=adam,metrics=['accuracy'])
    class_weight = {1 :26,
          0:1}

    callbacks = [
                roc_auc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test)),  # call this before EarlyStopping
                EarlyStopping(monitor='norm_gini_val', patience=3, mode='max', verbose=1),
             CSVLogger('keras-5fold-run-01-v1-epochs.log', separator=',', append=False),
                ModelCheckpoint(
                        'keras-5fold-run-01-v1-fold-' + str('%02d' % (k + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check',
                        monitor='norm_gini_val', mode='max', # mode must be set to max or Keras will be confused
                        save_best_only=True,
                        verbose=1)
            ]

    history = model.fit(X_train, y_train,
                            batch_size=batch_size,
                            class_weight = class_weight,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(X_test,y_test),callbacks=callbacks)
    del model
    model = keras.models.load_model('keras-5fold-run-01-v1-fold-' + str('%02d' % (k + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check')
    score = model.evaluate(X_test, y_test, verbose=0)
        #model_name = '/home/santanu/Downloads/Insurance Claim/' + 'nn_model__t_' + str(k) 
    model_name = '/home/santanu/Downloads/Insurance Claim/' + 'nn_model__tt_an' + str(k) 

    model.save(model_name)

    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    pred = model.predict(X_test,batch_size=5000)
    pred_nn = pred_nn + pred[:,0].tolist()
    pred = model.predict(test_x1,batch_size=5000)
    df_pred[str(k)] = pred[:,0].tolist()
    #print(eval_gini(y_test,pred[:,0].tolist()))
         


pred_df['nn'] = pred_nn
pred_df.to_csv('/home/santanu/Downloads/Insurance Claim/val_pred__ttt_an.csv')
pred_new = np.mean(df_pred[['1','2','3','4','5']].values,axis=1)
df_pred['target'] = pred_new
test_1 = pd.read_csv("/home/santanu/Downloads/Insurance Claim/test.csv")
df_pred['id'] = test_1['id']
df_pred[['id','target']].to_csv("/home/santanu/Downloads/Insurance Claim/test_subtt_an.csv",index=False)





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 400)               90000     
_________________________________________________________________
p_re_lu_28 (PReLU)           (None, 400)               400       
_________________________________________________________________
batch_normalization_28 (Batc (None, 400)               1600      
_________________________________________________________________
dropout_28 (Dropout)         (None, 400)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 200)               80200     
_________________________________________________________________
p_re_lu_29 (PReLU)           (None, 200)               200       
_________________________________________________________________
batch_normalization_29 (Batc (None, 200)               800       
__________

In [40]:
## Bit of more experimentation

df_pred = pd.DataFrame()
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn import preprocessing
import tensorflow
import keras
from keras.datasets import mnist
from keras.models import Sequential
#from keras.layers import Dense, Dropout,PReLU
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback
from sklearn.metrics import log_loss, roc_auc_score
import theano
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import custom_object_scope
from keras import callbacks

from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

class roc_auc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.x, verbose=0)
        roc = roc_auc_score(self.y, y_pred)
        logs['roc_auc'] = roc_auc_score(self.y, y_pred)
        logs['norm_gini'] = ( roc_auc_score(self.y, y_pred) * 2 ) - 1

        y_pred_val = self.model.predict_proba(self.x_val, verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        logs['roc_auc_val'] = roc_auc_score(self.y_val, y_pred_val)
        logs['norm_gini_val'] = ( roc_auc_score(self.y_val, y_pred_val) * 2 ) - 1

        print('\rroc_auc: %s - roc_auc_val: %s - norm_gini: %s - norm_gini_val: %s' % (str(round(roc,5)),str(round(roc_val,5)),str(round((roc*2-1),5)),str(round((roc_val*2-1),5))), end=10*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

  

train = pd.read_csv("/home/santanu/Downloads/Insurance Claim/train.csv")
train_y1 = train['target'].values
pred_nn = []
pred_df = pd.DataFrame()
#k = 0 
k = 0 
run = 0

kf = KFold(n_splits=5, random_state=0, shuffle=True)
for train_index, test_index in kf.split(train_x1):
    k += 1 
    test_df = test.copy()
    X_train,X_test = train_x1[train_index], train_x1[test_index]
    y_train, y_test = train_y1[train_index], train_y1[test_index]
    
    batch_size = 4096
    epochs = 300
    model = Sequential()
        #256
    model.add(Dense(512,input_shape=(224,)))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
        #256
    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    #sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) 
    model.compile(loss='binary_crossentropy', optimizer=adam,metrics=['accuracy'])
    class_weight = {1 :13,
          0:1}

    callbacks = [
                roc_auc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test)),  # call this before EarlyStopping
                EarlyStopping(monitor='norm_gini_val', patience=3, mode='max', verbose=1),
             CSVLogger('keras-5fold-run-01-v1-epochs.log', separator=',', append=False),
                ModelCheckpoint(
                        'keras-5fold-run-01-v1-fold-' + str('%02d' % (k + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check',
                        monitor='norm_gini_val', mode='max', # mode must be set to max or Keras will be confused
                        save_best_only=True,
                        verbose=1)
            ]

    history = model.fit(X_train, y_train,
                            batch_size=batch_size,
                            class_weight = class_weight,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(X_test,y_test),callbacks=callbacks)
    del model
    model = keras.models.load_model('keras-5fold-run-01-v1-fold-' + str('%02d' % (k + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check')
    score = model.evaluate(X_test, y_test, verbose=0)
        #model_name = '/home/santanu/Downloads/Insurance Claim/' + 'nn_model__t_' + str(k) 
    model_name = '/home/santanu/Downloads/Insurance Claim/' + 'nn_model__tt_an' + str(k) 

    model.save(model_name)

    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    pred = model.predict(X_test,batch_size=5000)
    pred_nn = pred_nn + pred[:,0].tolist()
    pred = model.predict(test_x1,batch_size=5000)
    df_pred[str(k)] = pred[:,0].tolist()
    #print(eval_gini(y_test,pred[:,0].tolist()))
         


pred_df['nn'] = pred_nn
pred_df.to_csv('/home/santanu/Downloads/Insurance Claim/val_pred__ttt_an1.csv')
pred_new = np.mean(df_pred[['1','2','3','4','5']].values,axis=1)
df_pred['target'] = pred_new
test_1 = pd.read_csv("/home/santanu/Downloads/Insurance Claim/test.csv")
df_pred['id'] = test_1['id']
df_pred[['id','target']].to_csv("/home/santanu/Downloads/Insurance Claim/test_subtt_an1.csv",index=False)





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_104 (Dense)            (None, 512)               115200    
_________________________________________________________________
p_re_lu_77 (PReLU)           (None, 512)               512       
_________________________________________________________________
batch_normalization_77 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_77 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_105 (Dense)            (None, 256)               131328    
_________________________________________________________________
p_re_lu_78 (PReLU)           (None, 256)               256       
_________________________________________________________________
batch_normalization_78 (Batc (None, 256)               1024      
__________